# 18 • Herramientas de texto

En este notebook se revisará distintas herramientas para realizar preproceso y análisis de texto.

## Contenido
1. Strings como lista de caracteres
2. Preproceso de texto
3. Regular expressions (*Regex*)
4. spaCy
5. Referencias  

## 1. Strings como lista de caracteres
Anteriormente ya habían trabajado con strings, sin embargo, podemos tratar a un string como una lista de caracteres. Además, el objeto 

In [1]:
texto = "Mi villano favorito"

# También se puede declarar así:
# texto = str("Mi villano favorito")

In [2]:
# Imprime un Carácter por renglón
for i in texto:
    print(i)

M
i
 
v
i
l
l
a
n
o
 
f
a
v
o
r
i
t
o


Existen algunas funciones propias de los *strings* que nos ayudarán para trabajar con textos, como:
- **lower**, pone la cadena en minúsculas
- **upper**, pone la cadena en mayúsculas
- **title**, pone en mayúscula la primera letra de cada palabra de un enunciado
- **replace**, para sustituir algún texto
- **rfind**, encuentra dentro de una cadena dónde inicia un texto de nuestro interés
- **rspilt**, separa un enunciado en palabras (*tokens*)

In [3]:
# lower
texto.lower()

'mi villano favorito'

In [4]:
# upper
texto.upper()

'MI VILLANO FAVORITO'

In [5]:
# title
texto.title()

'Mi Villano Favorito'

In [6]:
# replace
texto.replace("villano", "marciano")

'Mi marciano favorito'

In [7]:
# rfind
texto.rfind("villano")

3

In [8]:
# split
texto.rsplit()

['Mi', 'villano', 'favorito']

## 2. Preproceso de texto
Usualmente la información en formato de texto se encuentra desordenada y sin estructura. Para poder realizar un mejor análisis es importante "normalizar" esta información.

Existen distintas herramientas para el preproceso de datos que podemos utilizar para obtener una base de datos limpia, entre las que se encuentran: utilizar letras minúscula, lematización (usar la raíz de la palabra), separa un enunciado en *tokens*, remover puntuación y *stopwords*, entre otras.

In [9]:
# load libraries
import pandas as pd
import numpy as np
import snscrape.modules.twitter as sntwitter

### Base de datos inicial (raw)
En este caso se consultaron 1,000 tweets con el hashtag #ElINENoSeToca

In [10]:
%%time

# Parámetros
tweets_list_ine = []
maxTweets_ine = 1_000
date_initial = "2022-10-01"

# Get tweets
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('#ElINENoSeToca').get_items()): # se puede añadir esto --> since:'+date_initial
        if i>maxTweets_ine-1:
            break
        tweets_list_ine.append([tweet.user.username, tweet.date, tweet.content])
        
# Pandas dataframe con tweets que mencionen el hashtag #ElINENoSeToca
column_names = ("username","date","tweet")
df = pd.DataFrame(tweets_list_ine, columns=column_names)
#df['day'] = df['date'].dt.strftime('%Y-%m-%d')

CPU times: user 522 ms, sys: 49.2 ms, total: 571 ms
Wall time: 27.9 s


In [11]:
# así se ve la base de datos
df.head()

,username,date,tweet
0,fer_hernandez71,2022-11-16 04:17:17+00:00,@_VicenteSerrano @lopezobrador_ Que vayan pens...
1,1927March,2022-11-16 04:17:16+00:00,@proceso Pues ya pueden ir denunciando al seño...
2,ralR5,2022-11-16 04:15:58+00:00,¡Quedaron! @lorenzocordovav @CiroMurayamaINE #...
3,mgrminet,2022-11-16 04:15:46+00:00,@TapiaFernanda Tienes que defender el chayote ...
4,TarangoBernardo,2022-11-16 04:14:48+00:00,Yo también estuve ahí y coincido... somos un c...


In [12]:
# descripción general de los datos
df.describe()

/var/folders/fs/sshbhn_57gv4s9wnsggc69_00000gn/T/ipykernel_36401/2951990929.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe()


,username,date,tweet
count,1000,1000,1000
unique,625,974,986
top,seppruiz,2022-11-16 01:09:33+00:00,@PartidoMorenaMx #ElINENoSeToca
freq,18,2,5
first,NaN,2022-11-15 23:04:22+00:00,NaN
last,NaN,2022-11-16 04:17:17+00:00,NaN


In [13]:
# identificación de valores únicos por variable
df.nunique()

username    625
date        974
tweet       986
dtype: int64

### Eliminar signos de puntuación
Aquí se eliminan los signos de puntuación con excepción de `#` para poder identificar los hashtags

In [14]:
#library that contains punctuation
import string
signos_puntuacion = string.punctuation
signos_puntuacion = signos_puntuacion.replace("#", "")  # quitar '#' de hashtags 
signos_puntuacion = signos_puntuacion.replace("@", "")  # quitar '@' de cuentas
signos_puntuacion

'!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~'

In [15]:
#defining the function to remove punctuation
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in signos_puntuacion])
    return punctuationfree

#storing the puntuation free text
df['tweet_clean'] = df['tweet'].apply(lambda x:remove_punctuation(x))
df.head()

,username,date,tweet,tweet_clean
0,fer_hernandez71,2022-11-16 04:17:17+00:00,@_VicenteSerrano @lopezobrador_ Que vayan pens...,@VicenteSerrano @lopezobrador Que vayan pensan...
1,1927March,2022-11-16 04:17:16+00:00,@proceso Pues ya pueden ir denunciando al seño...,@proceso Pues ya pueden ir denunciando al seño...
2,ralR5,2022-11-16 04:15:58+00:00,¡Quedaron! @lorenzocordovav @CiroMurayamaINE #...,¡Quedaron @lorenzocordovav @CiroMurayamaINE #E...
3,mgrminet,2022-11-16 04:15:46+00:00,@TapiaFernanda Tienes que defender el chayote ...,@TapiaFernanda Tienes que defender el chayote ...
4,TarangoBernardo,2022-11-16 04:14:48+00:00,Yo también estuve ahí y coincido... somos un c...,Yo también estuve ahí y coincido somos un chin...


### Textos en minúsculas
Parte de la normalización es poner todos los textos en minúsculas (o mayúsculas), esto puede servir posteriormente por ejemplo para hacer un conteo de palabras.

⚠️ _**Nota**: Al hacer esto existe el riesgo que se pierda información contextual; por ejemplo, dentro de un tweet cuando alguien escribe una palabra específica en mayúsculas y las demás en minúsculas esto conlleva un mayor importancia o aumento en tono de voz._

In [16]:
# cambiar los tweets a letras minúsculas
df['tweet_clean'] = df['tweet_clean'].apply((lambda x: x.lower()))
df.head()

,username,date,tweet,tweet_clean
0,fer_hernandez71,2022-11-16 04:17:17+00:00,@_VicenteSerrano @lopezobrador_ Que vayan pens...,@vicenteserrano @lopezobrador que vayan pensan...
1,1927March,2022-11-16 04:17:16+00:00,@proceso Pues ya pueden ir denunciando al seño...,@proceso pues ya pueden ir denunciando al seño...
2,ralR5,2022-11-16 04:15:58+00:00,¡Quedaron! @lorenzocordovav @CiroMurayamaINE #...,¡quedaron @lorenzocordovav @ciromurayamaine #e...
3,mgrminet,2022-11-16 04:15:46+00:00,@TapiaFernanda Tienes que defender el chayote ...,@tapiafernanda tienes que defender el chayote ...
4,TarangoBernardo,2022-11-16 04:14:48+00:00,Yo también estuve ahí y coincido... somos un c...,yo también estuve ahí y coincido somos un chin...


### Tokenization
Aquí veremos cómo separar los enunciados (*tweets*) en *tokens* (palabras o cadenas de caracteres).

In [17]:
# getting the tokens per tweet
df['tweet_clean'] = df['tweet_clean'].apply(lambda x: x.rsplit())
df.head()

,username,date,tweet,tweet_clean
0,fer_hernandez71,2022-11-16 04:17:17+00:00,@_VicenteSerrano @lopezobrador_ Que vayan pens...,"[@vicenteserrano, @lopezobrador, que, vayan, p..."
1,1927March,2022-11-16 04:17:16+00:00,@proceso Pues ya pueden ir denunciando al seño...,"[@proceso, pues, ya, pueden, ir, denunciando, ..."
2,ralR5,2022-11-16 04:15:58+00:00,¡Quedaron! @lorenzocordovav @CiroMurayamaINE #...,"[¡quedaron, @lorenzocordovav, @ciromurayamaine..."
3,mgrminet,2022-11-16 04:15:46+00:00,@TapiaFernanda Tienes que defender el chayote ...,"[@tapiafernanda, tienes, que, defender, el, ch..."
4,TarangoBernardo,2022-11-16 04:14:48+00:00,Yo también estuve ahí y coincido... somos un c...,"[yo, también, estuve, ahí, y, coincido, somos,..."


### Eliminar *stopwords*
Las *stopwords* son palabras de uso común que no añaden un valor adicional al análisis de texto o que no tienen sentido, por lo que pueden ser eliminadas de nuestra lista de tokens.

Existen distintas maneras de obtener la lista de stopwords:  
> (1) Crear nuestra propia lista  

> (2) Buscar alguna base de datos que contenga la lista de stopwords
> - En GitHub encontré una lista de stopwords el el repositorio [Alir3z4/stop-words](https://github.com/Alir3z4/stop-words/blob/master/spanish.txt)
> - Una alternativa es una lista de Kaggle [Spanish Stopwords W2V](https://www.kaggle.com/code/mpwolke/spanish-stopwords-w2v)
> - Otra alternativa es la lista que da la página [countwordsfree.com](https://countwordsfree.com/stopwords/spanish)

> (3) Algo más estandarizado y completo es utilizar la librería de *Natural Language Toolkit* (**NLTK**), la cual tiene listas de stopwords en distintos idiomas, donde (además de inglés) podemos encontrar la lista de palabras en español. Para instalar este paquete así como mayor detalle de uso, revisa la página de la [librería NLTK](https://www.nltk.org/data.html).

⚠️ _**Nota**: antes de usar alguna de las opciones anteriores deberás de pensar cuál lista de stopwords se acopla mejor a tus necesidades!_

In [18]:
# (1) Crear nuestra propia lista de stopwords
stopwords_1 = ['a','ante','bajo','cabe','con','no','de','del','al','tras']

print("Número de stopwords en mi lista (1):", len(stopwords_1))
stopwords_1

Número de stopwords en mi lista (1): 10


['a', 'ante', 'bajo', 'cabe', 'con', 'no', 'de', 'del', 'al', 'tras']

In [19]:
# (2) Buscar alguna base de datos que contenga la lista de stopwords.
#     En este ejemplo bajamos la lista de GitHub
import urllib3

http = urllib3.PoolManager()
r = http.request('GET', "https://raw.githubusercontent.com/Alir3z4/stop-words/master/spanish.txt")
stopwords_2 = r.data.decode('utf-8').replace("\n"," ").rsplit()

print("Número de stopwords en mi lista (2):", len(stopwords_2))
stopwords_2[:15]

Número de stopwords en mi lista (2): 609


['a',
 'actualmente',
 'adelante',
 'además',
 'afirmó',
 'agregó',
 'ahora',
 'ahí',
 'al',
 'algo',
 'alguna',
 'algunas',
 'alguno',
 'algunos',
 'algún']

In [25]:
# (3) Uso de NLP Toolkit library

## Para instalar la librería revisa la página: https://www.nltk.org/data.html
## Nota, este proceso puede ser tardado...
# !pip install nltk
# nltk.download()

# load library
import nltk

stopwords_3 = nltk.corpus.stopwords.words('spanish')

print("Número de stopwords en de NLTK library (3):", len(stopwords_3))
stopwords_3[:15]

Número de stopwords en de NLTK library (3): 313


['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con']

**Vamos a eliminar las stopwords usando la lista que obtivimos de NLTK library (opción 3):**

In [26]:
#defining the function to remove stopwords from tokenized text
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords_3]
    return output

#applying the stopwords function to thet tweets
df['tweet_clean'] = df['tweet_clean'].apply(lambda x:remove_stopwords(x))
df.head()

,username,date,tweet,tweet_clean
0,fer_hernandez71,2022-11-16 04:17:17+00:00,@_VicenteSerrano @lopezobrador_ Que vayan pens...,"[@vicenteserrano, @lopezobrador, vayan, pensan..."
1,1927March,2022-11-16 04:17:16+00:00,@proceso Pues ya pueden ir denunciando al seño...,"[@proceso, pues, pueden, ir, denunciando, seño..."
2,ralR5,2022-11-16 04:15:58+00:00,¡Quedaron! @lorenzocordovav @CiroMurayamaINE #...,"[¡quedaron, @lorenzocordovav, @ciromurayamaine..."
3,mgrminet,2022-11-16 04:15:46+00:00,@TapiaFernanda Tienes que defender el chayote ...,"[@tapiafernanda, defender, chayote, dan, #elin..."
4,TarangoBernardo,2022-11-16 04:14:48+00:00,Yo también estuve ahí y coincido... somos un c...,"[ahí, coincido, chingo, #elinenosetoca, #eline..."


### Lematización
Este paso nos servirá para extraer la la raíz de las palabras, para poder "normalizar" el texto y realizar un mejor análisis. Aquí nuevamente utilizaremos la librería `NLTK`.

In [27]:
#defining the object for Lemmatization
wordnet_lemmatizer = nltk.stem.WordNetLemmatizer()

#defining the function for lemmatization
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text

#lemmatization of our tweets
df['tweet_clean'] = df['tweet_clean'].apply(lambda x:lemmatizer(x))
df.head()

,username,date,tweet,tweet_clean
0,fer_hernandez71,2022-11-16 04:17:17+00:00,@_VicenteSerrano @lopezobrador_ Que vayan pens...,"[@vicenteserrano, @lopezobrador, vayan, pensan..."
1,1927March,2022-11-16 04:17:16+00:00,@proceso Pues ya pueden ir denunciando al seño...,"[@proceso, pues, pueden, ir, denunciando, seño..."
2,ralR5,2022-11-16 04:15:58+00:00,¡Quedaron! @lorenzocordovav @CiroMurayamaINE #...,"[¡quedaron, @lorenzocordovav, @ciromurayamaine..."
3,mgrminet,2022-11-16 04:15:46+00:00,@TapiaFernanda Tienes que defender el chayote ...,"[@tapiafernanda, defender, chayote, dan, #elin..."
4,TarangoBernardo,2022-11-16 04:14:48+00:00,Yo también estuve ahí y coincido... somos un c...,"[ahí, coincido, chingo, #elinenosetoca, #eline..."


### Eliminar URLS
En este caso, también eliminamos las URLs.

In [28]:
def erase_urls(text):
    output = [i for i in text if i.rfind("http")==-1]
    return output

# texto.rfind("villano")
df['tweet_clean'] = df['tweet_clean'].apply(lambda x:erase_urls(x))
df.head()

,username,date,tweet,tweet_clean
0,fer_hernandez71,2022-11-16 04:17:17+00:00,@_VicenteSerrano @lopezobrador_ Que vayan pens...,"[@vicenteserrano, @lopezobrador, vayan, pensan..."
1,1927March,2022-11-16 04:17:16+00:00,@proceso Pues ya pueden ir denunciando al seño...,"[@proceso, pues, pueden, ir, denunciando, seño..."
2,ralR5,2022-11-16 04:15:58+00:00,¡Quedaron! @lorenzocordovav @CiroMurayamaINE #...,"[¡quedaron, @lorenzocordovav, @ciromurayamaine..."
3,mgrminet,2022-11-16 04:15:46+00:00,@TapiaFernanda Tienes que defender el chayote ...,"[@tapiafernanda, defender, chayote, dan, #elin..."
4,TarangoBernardo,2022-11-16 04:14:48+00:00,Yo también estuve ahí y coincido... somos un c...,"[ahí, coincido, chingo, #elinenosetoca, #eline..."


### Resultado final de preproceso (BD final)
A continuación se muestran los textos iniciales en la columna "tweets", y la información preprocesada en la columna "tweet_clean", a manera de comparación.

In [29]:
df[['tweet', 'tweet_clean']]

,tweet,tweet_clean
0,@_VicenteSerrano @lopezobrador_ Que vayan pens...,"[@vicenteserrano, @lopezobrador, vayan, pensan..."
1,@proceso Pues ya pueden ir denunciando al seño...,"[@proceso, pues, pueden, ir, denunciando, seño..."
2,¡Quedaron! @lorenzocordovav @CiroMurayamaINE #...,"[¡quedaron, @lorenzocordovav, @ciromurayamaine..."
3,@TapiaFernanda Tienes que defender el chayote ...,"[@tapiafernanda, defender, chayote, dan, #elin..."
4,Yo también estuve ahí y coincido... somos un c...,"[ahí, coincido, chingo, #elinenosetoca, #eline..."
...,...,...
995,@alitomorenoc Tienes en tus manos reivindicart...,"[@alitomorenoc, manos, reivindicarte, ciudadan..."
996,@alitomorenoc Tienes en tus manos reivindicart...,"[@alitomorenoc, manos, reivindicarte, ciudadan..."
997,#ElINENoSeToca https://t.co/XLx5KoLJPP,[#elinenosetoca]
998,#YoSiFuiALaMarcha #ElINENoSeToca https://t.co/...,"[#yosifuialamarcha, #elinenosetoca]"


In [137]:
print("Ejemplo de tweet...\n- original tweet:\n",df['tweet'][1],
      "\n\n- tweet clean:\n", df['tweet_clean'][1])

Ejemplo de tweet...
- original tweet:
 @proceso Pues ya pueden ir denunciando al señor @MARIACLEMENTEMX por los actos de acoso, insultos, discriminación en agravio de personas que acudimos a la marcha #ElINENoSeToca . Este fulano se cree intocable. 

- tweet clean:
 ['@proceso', 'pues', 'pueden', 'ir', 'denunciando', 'señor', '@mariaclementemx', 'actos', 'acoso', 'insultos', 'discriminación', 'agravio', 'persona', 'acudimos', 'marcha', '#elinenosetoca', 'fulano', 'cree', 'intocable']


## 3. Regular expressions (*Regex*)
Las expresiones regulares mejor conocidas como *regex* son una herramienta que nos sirve para trabajar con texto, con el cual se pueden realziar distintas operaciones como búsqueda o reemplazo de un texto específico.

Existen distintos operadores que se utilizan en *regex*, estos son los que utilizaremos para nuestros ejemplos. 


|Operador|Descripción (_en inglés_)|
|:------:|:----------------------|
|. (punto)|In the default mode, this matches any character except a newline. If the DOTALL flag has been specified, this matches any character including a newline.|
|* (asterisco) |Causes the resulting RE to match 0 or more repetitions of the preceding RE, as many repetitions as are possible. ab* will match ‘a’, ‘ab’, or ‘a’ followed by any number of ‘b’s.|
|+ (más) |Causes the resulting RE to match 1 or more repetitions of the preceding RE. ab+ will match ‘a’ followed by any non-zero number of ‘b’s; it will not match just ‘a’.|
|[ ]| Used to indicate a set of characters. In a set:<ul><li>Characters can be listed individually, e.g. [amk] will match 'a', 'm', or 'k'.</li><li>Ranges of characters can be indicated by giving two characters and separating them by a '-', for example [a-z] will match any lowercase ASCII letter, [0-5][0-9] will match all the two-digits numbers from 00 to 59, and [0-9A-Fa-f] will match any hexadecimal digit. If - is escaped (e.g. [a\-z]) or if it’s placed as the first or last character (e.g. [-a] or [a-]), it will match a literal '-'.</li><li>Special characters lose their special meaning inside sets. For example, [(+*)] will match any of the literal characters '(', '+', '*', or ')'.</li></ul>|
|\n |line break|

⚠️ Se recomienda ampliamente que revisen la página de la [librería **`re`** - *Regular expression operations*](https://docs.python.org/3/library/re.html) para identificar todos los operadores, documentación y ejemplos de como utilizar _regex_.

📌 Si les interesa practicar el uso de _regex_, les recomiendo la página de [regexr.com](https://regexr.com)

In [108]:
# load regex library
import re

In [123]:
# Extracto de canción "El amor" de Arjona, link: https://www.letras.com/arjona-ricardo/1963651/
texto = """El amor tiene firma de autor
En las causas pérdidas
El amor siempre empieza soñando
Y termina en insomnio
Es un acto profundo de fe
Que huele a mentira
El amor baila al son que le toquen
Sea Dios o el demonio
Sea Dios o el demonio

El amor es la guerra pérdida
Entre el sexo y la risa
Es la llave con que abres
El grifo del agua en los ojos
Es el tiempo más lento del mundo
Cuando va de prisa
El amor se abre paso despacio
No importa el cerrojo

El amor es la arrogancia
De aferrarse a lo imposible
Es buscar en otra parte
Lo que no encuentras en ti
"""

### Encontrar un texto usando _regex_

In [124]:
# Encontrar todos los enunciados que tengan la palabra 'amor'
re.findall(r'.*[A|a]mor.*', texto)

['El amor tiene firma de autor',
 'El amor siempre empieza soñando',
 'El amor baila al son que le toquen',
 'El amor es la guerra pérdida',
 'El amor se abre paso despacio',
 'El amor es la arrogancia']

#### Sustituir una palabra, frase o cadena de caracterés por otro texto

In [127]:
# Substituir la palabra 'amor' por 'cepillo de dientes'
texto_sub = re.sub('[A|a]mor', 'CePiLLo dE DiEnTeS', texto)  # cambia la palabra
texto_sub = re.sub('\n+', '. ', texto_sub)  # cambia el salto de dos renglones por un punto 
texto_sub

'El CePiLLo dE DiEnTeS tiene firma de autor. En las causas pérdidas. El CePiLLo dE DiEnTeS siempre empieza soñando. Y termina en insomnio. Es un acto profundo de fe. Que huele a mentira. El CePiLLo dE DiEnTeS baila al son que le toquen. Sea Dios o el demonio. Sea Dios o el demonio. El CePiLLo dE DiEnTeS es la guerra pérdida. Entre el sexo y la risa. Es la llave con que abres. El grifo del agua en los ojos. Es el tiempo más lento del mundo. Cuando va de prisa. El CePiLLo dE DiEnTeS se abre paso despacio. No importa el cerrojo. El CePiLLo dE DiEnTeS es la arrogancia. De aferrarse a lo imposible. Es buscar en otra parte. Lo que no encuentras en ti. '

### Dividir el texto en una lista de strings

In [140]:
re.split("\n+", texto)

['El amor tiene firma de autor',
 'En las causas pérdidas',
 'El amor siempre empieza soñando',
 'Y termina en insomnio',
 'Es un acto profundo de fe',
 'Que huele a mentira',
 'El amor baila al son que le toquen',
 'Sea Dios o el demonio',
 'Sea Dios o el demonio',
 'El amor es la guerra pérdida',
 'Entre el sexo y la risa',
 'Es la llave con que abres',
 'El grifo del agua en los ojos',
 'Es el tiempo más lento del mundo',
 'Cuando va de prisa',
 'El amor se abre paso despacio',
 'No importa el cerrojo',
 'El amor es la arrogancia',
 'De aferrarse a lo imposible',
 'Es buscar en otra parte',
 'Lo que no encuentras en ti',
 '']

## 4. spaCy

In [143]:
# # Install spaCy webpage: https://spacy.io/usage
# ! conda install -c conda-forge spacy  # <-install spaCy
# ! python -m spacy download en_core_web_sm   # <-spaCy English pipeline optimized for CPU.
# ! python -m spacy download es_core_news_sm  # <-spaCy Spanish pipeline optimized for CPU.

In [211]:
# load library
import spacy

### Obtener características de cada *token*

In [212]:
# load spaCy Spanish pipeline optimized
nlp = spacy.load("es_core_news_sm")

# ejemplo
enunciado = 'Me llegó un email al correo leo234@gmail.com que dice: "OMG! El señor Carlos rompió el control del televisor hoy y tuve que \
comprar otro en www.amazon.com.mx".'

# volver el enunciado un objeto tipo NLP
doc = nlp(enunciado) #(sentences[0])
print("Enunciado:\n",doc.text,"\n")

# Para cada token (palabra/cadena) se obtienen distintas características:
# - texto
# - tag con tipo de texto: noun, verb, determinant, adverb, conjunction
# - dep: dependenxy
# - like_email: identifica si el token es un email
# - like_url: identifica si el token es una página web
list_tokens = []
for token in doc:
    # print(token.text, token.pos_, token.dep_, token.like_email)
    list_tokens.append([token.text,token.lemma_,token.pos_,token.dep_,token.like_email,
                        token.like_url, token.head.text])

# Revisar algunas de las características de cada token
pd.DataFrame(list_tokens, columns=['text','lemma','pos','dep','is_email','is_url','next_text'])

Enunciado:
 Me llegó un email al correo leo234@gmail.com que dice: "OMG! El señor Carlos rompió el control del televisor hoy y tuve que comprar otro en www.amazon.com.mx". 



,text,lemma,pos,dep,is_email,is_url,next_text
0,Me,yo,PRON,iobj,False,False,llegó
1,llegó,llegar,VERB,ROOT,False,False,llegó
2,un,uno,DET,det,False,False,email
3,email,email,NOUN,nsubj,False,False,llegó
4,al,al,ADP,case,False,False,correo
5,correo,correo,NOUN,obl,False,False,llegó
6,leo234@gmail.com,leo234@gmail.com,NUM,appos,True,False,correo
7,que,que,PRON,nsubj,False,False,dice
8,dice,decir,VERB,acl,False,False,correo
9,:,:,PUNCT,punct,False,False,OMG


### Preproceso con spaCy
Ejemplo de función para preproceso usando spaCy, Regex y métodos de strings, comparando resultados con el del preproceso que se vio al inicio de este Jupyter notebook.

In [240]:
def preprocess(text,
            min_token_len = 2,
            irrelevant_pos = ['PRON', 'SPACE', 'PUNCT', 'ADV', 
                              'ADP', 'CCONJ', 'AUX', 'PRP'],
            avoid_entities = ['PERSON', 'ORG', 'LOC', 'GPE']):
    # note: Didn't use the following options in the `preprocess_comments`
    #    - 'PROPN', erase proper names, but also words as orange.
    #    - 'DET', removes the word 'no', which changes the meaning.
    """
    Function that identify sensible information, anonymize and transforms
    the data in a useful format for using with tokens.
    Parameters
    -------------
    text : (list)
        the list of text to be preprocessed
    irrelevant_pos : (list)
        a list of irrelevant 'pos' tags
    avoid_entities : (list)
        a list of entity labels to be avoided

    Returns
    -------------
    (list) list of preprocessed text

    Example
    -------------
    example = ["Hello, I'm George and I love swimming!",
                "I am a really good cook; what about you?",
                "Contact me at george23@gmail.com"]
    preprocess(example)
    (output:) ['hello love swimming', 'good cook', 'contact']
    """
    result = []
    others = ["'s", "the", "that", "this", "to", "-PRON-"]
    # comment: "-PRON-" is a lemma for "my", "your", etc.

    # function
    for sent in text:
        sent = str(sent).lower()
        sent = re.sub(r"(F|f)acebook", "social media", sent)
        sent = re.sub(r"(T|t)witter", "social media", sent)
        sent = re.sub(r"(I|i)nstagram", "social media", sent)

        result_sent = []
        doc = nlp(sent)
        entities = [str(ent) for ent in doc.ents if ent.label_ in avoid_entities]
        # This helps to detect names of persons, organization and dates

        for token in doc:            
            if (token.like_email or
                token.like_url or
                token.pos_ in irrelevant_pos or
                str(token) in entities or
                str(token.lemma_) in others or
                len(token) < min_token_len):
                continue
            else:
                result_sent.append(token.lemma_)
        result.append(result_sent)
    return result

In [241]:
# Preprocess tweets
df['tweet_spacy_preprocess'] = preprocess(df['tweet'])

In [243]:
df[['tweet','tweet_clean','tweet_spacy_preprocess']]

,tweet,tweet_clean,tweet_spacy_preprocess
0,@_VicenteSerrano @lopezobrador_ Que vayan pens...,"[@vicenteserrano, @lopezobrador, vayan, pensan...","[@_vicenteserrano, @lopezobrador, pensar, el, ..."
1,@proceso Pues ya pueden ir denunciando al seño...,"[@proceso, pues, pueden, ir, denunciando, seño...","[@proceso, pues, denunciar, señor, @mariacleme..."
2,¡Quedaron! @lorenzocordovav @CiroMurayamaINE #...,"[¡quedaron, @lorenzocordovav, @ciromurayamaine...","[quedar, @lorenzocordovav, @ciromurayamaine, e..."
3,@TapiaFernanda Tienes que defender el chayote ...,"[@tapiafernanda, defender, chayote, dan, #elin...","[@tapiafernandar, tener, que, defender, el, ch..."
4,Yo también estuve ahí y coincido... somos un c...,"[ahí, coincido, chingo, #elinenosetoca, #eline...","[estar, coincido, uno, chingo, elinesoyyoerest..."
...,...,...,...
995,@alitomorenoc Tienes en tus manos reivindicart...,"[@alitomorenoc, manos, reivindicarte, ciudadan...","[@alitomorenoc, tener, tu, mano, reivindicarte..."
996,@alitomorenoc Tienes en tus manos reivindicart...,"[@alitomorenoc, manos, reivindicarte, ciudadan...","[@alitomorenoc, tener, tu, mano, reivindicarte..."
997,#ElINENoSeToca https://t.co/XLx5KoLJPP,[#elinenosetoca],[]
998,#YoSiFuiALaMarcha #ElINENoSeToca https://t.co/...,"[#yosifuialamarcha, #elinenosetoca]","[yosifuialamarcha, elinenosetoca]"


## 5. References
- [Python String Methods](https://docs.python.org/3/library/stdtypes.html) in Python Documentation.
- [Text Preprocessing in NLP with Python codes](https://www.analyticsvidhya.com/blog/2021/06/text-preprocessing-in-nlp-with-python-codes/) by Deepanshi, in Analytics Vidhya.
- Librería de Python [*Natural Language Toolkit* - `NLTK`](https://www.nltk.org/index.html)
- Librería de Python [*Regular expression operations* - `re`](https://docs.python.org/3/library/re.html)
- Python script [preprocess.py](https://github.com/vcuspinera/UBC_MDS_Capstone_BCStats/blob/master/src/data/preprocess.py) del Capstone project de UBC MDS y BC Stats, por Sukriti Trehan, Karanpal Singh, Carlina Kim y Victor Cuspinera.